In [ ]:
def moving_average_smooth(data, window_size=3):
    """
    Smooths a 2D array along the first axis using a moving average.

    Parameters:
        data (ndarray): 2D array to be smoothed.
        window_size (int): Size of the moving average window.

    Returns:
        ndarray: Smoothed 2D array.
    """
    smoothed_data = np.apply_along_axis(
        lambda x: np.convolve(x, np.ones(window_size)/window_size, mode='same'),
        axis=0,
        arr=data
    )
    return smoothed_data

def bin_average(data, bin_size=2):
    """
    Bins a 2D array along the first axis and averages within each bin.

    Parameters:
        data (ndarray): 2D array to be binned and averaged.
        bin_size (int): Number of data points to bin together.

    Returns:
        ndarray: Binned and averaged 2D array.
    """
    # Ensure the number of rows is divisible by bin_size
    num_bins = data.shape[0] // bin_size
    trimmed_data = data[:num_bins * bin_size]  # Trim excess rows if not divisible

    # Reshape and average within bins
    binned_data = trimmed_data.reshape(num_bins, bin_size, -1).mean(axis=1)
    
    return binned_data

def rolling_bin_average(data, time_axis, bin_size=5):
    """
    Bins the data and time axis using a rolling average with the specified bin size.

    Parameters:
        data (ndarray): 1D array of data points to be averaged.
        time_axis (ndarray): 1D array of time points corresponding to the data.
        bin_size (int): Number of data points to bin together (default is 5).

    Returns:
        tuple: (binned_data, binned_time_axis)
            - binned_data: 1D array of binned (averaged) data points.
            - binned_time_axis: 1D array of binned (averaged) time points.
    """
    num_bins = len(data) // bin_size  # Number of complete bins
    
    # Trim data and time axis to fit into complete bins
    trimmed_data = data[:num_bins * bin_size]
    trimmed_time_axis = time_axis[:num_bins * bin_size]
    
    # Reshape data and time axis into (num_bins, bin_size) and average along axis=1
    binned_data = trimmed_data.reshape(num_bins, bin_size).mean(axis=1)
    binned_time_axis = trimmed_time_axis.reshape(num_bins, bin_size).mean(axis=1)
    
    return binned_data, binned_time_axis

In [ ]:
#data_smoothed = moving_average_smooth(data[frequency_index, :])
data_bin,time_trim = data_smoothed, time_axis #rolling_bin_average(data_smoothed, time_axis, 5)
#trimmed_time_axis = time_axis[:data_smoothed.size]
#print(trimmed_time_axis.size)
#print(data_smoothed.size)

In [ ]:
rel_time_start = 160 * 4
rel_time_stop = 180 * 4
intensity_values_rel_smooth = intensity_noGauss[rel_time_start:rel_time_stop]
time_axis_rel = time_trim[rel_time_start:rel_time_stop]
print(time_trim)
c = 299792458 # speed of light in m/s
B = 5.49 # length of Baseline in meters
lam = c/(chosen_frequency*1e6) # wavelength in meters

print(chosen_frequency)
print(lam)
print(time_axis_rel)

# Define the theoretical fit function
def theoretical_fit(x, S, omega, phi):
    return S * (1 + np.cos(2* np.pi *B * omega * x / lam + phi))

# Initial guesses for S, omega, and phi
initial_guess = [0.8, 0.004, 0]

# Set bounds for S, omega, and phi
lower_bounds = [0, 0, -np.pi]  # Lower bound for S, omega, and phi
upper_bounds = [3, 0.1, np.pi]  # Upper bound for S, omega, and phi

# Curve fitting with bounds
params, covariance = curve_fit(
    theoretical_fit,
    time_axis_rel,
    intensity_values_rel_smooth,
    p0=initial_guess,
    bounds=(lower_bounds, upper_bounds),
    maxfev=50000
)

print(params)

# Extract the fitted parameters
S_fit, omega_fit, phi_fit = params

# Generate fitted curve
fitted_curve = theoretical_fit(time_axis_rel, S_fit, omega_fit, phi_fit)



# Plot the intensity of the chosen frequency over time
plt.figure(figsize=(10, 4))
plt.plot(time_axis_rel, intensity_values_rel_smooth , label=f'Frequency {chosen_frequency} MHz')
plt.plot(time_axis_rel, fitted_curve, label='Fitted Curve', linewidth=2, color='red')
plt.xlabel('Time')
plt.ylabel('Intensity')
plt.title(f'Intensity vs Time for Frequency {chosen_frequency} MHz')
plt.legend()
plt.grid(True)
plt.show()